Homework 2: Discovery of Frequent Itemsets and Association Rules

In [45]:
# Discovery of Frequent Itemsets and Association Rules

# read dataset from T10I4D100K.dat
# read intergers sets from dataset line by line
def read_dataset(filename):
    dataset = []
    with open(filename, 'r') as f:
        for line in f:
            lines = list(map(int, line.strip().split()))
            # lines.sort()
            dataset.append(lines)
    # dataset.sort()
    return dataset

dataset = read_dataset('./T10I4D100K.dat')
# length of dataset
print('length of dataset:', len(dataset))

length of dataset: 100000


implement the A-Priori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions.

In [46]:
# get the apporopriate support threshold
# typicallly, support threshold is the length of dataset / 100
def get_support_threshold(dataset):
    return len(dataset) * 0.01

print('support threshold:', int(get_support_threshold(dataset)))

s = get_support_threshold(dataset)
# s = 3000

support threshold: 1000


In [47]:
# finding frequent itemsets of integers with support at least s in the dataset
import numpy as np
import pandas as pd
import itertools
class Apriori:
    def __init__(self, dataset, support_threshold):
        self.dataset = dataset
        self.support_threshold = support_threshold
        self.frequent_itemsets = []
        self.frequent_item = []
        self.all_freq_itemsets = []
        self.item_frequency = {}
        self.frequent_item_sets = []
        
   # compute the support of given itemset
    def support(self, itemset):
        support = 0
        for transaction in self.dataset:
            if set(itemset).issubset(set(transaction)):
                support += 1
        return support
    
    # filter generated possible itemset
    def filter_itemset(self, itemset):
        return (self.support(itemset) >= self.support_threshold)

    # get 1-item_set
    def get_1_item_set(self):
        for transaction in self.dataset:
            for item in transaction:
                if item not in self.item_frequency:
                    self.item_frequency[item] = 1
                else:
                    self.item_frequency[item] += 1
        for item in self.item_frequency:
            if self.item_frequency[item] >= self.support_threshold:
                self.frequent_item.append(item)
        # self.frequent_item.sort()
        return self.frequent_item

    # update the dataset
    def update_dataset(self, itemset):
        new_dataset = []
        for transaction in self.dataset:
            for item in transaction:
                if item in itemset:
                    new_dataset.append(transaction)
                    break
        # new_dataset.sort()
        self.dataset = new_dataset
        return self.dataset
    
    # get k-item_set
    # compute the support using the support function
    def get_k_item_set(self, _k_1_itemsets, k):
        k_itemsets = []
        possible_itemsets = list(itertools.combinations(_k_1_itemsets, k))
        # possible_itemsets.sort()
        for itemset in possible_itemsets:
            if self.filter_itemset(itemset):
                k_itemsets.append(itemset)
                print(f'{k}-itemset:', itemset, 'support:', self.support(itemset))
        return k_itemsets

    # get frequent itemset
    def get_frequent_itemset(self):
        _1_itemsets = self.get_1_item_set()
        self.frequent_itemsets.append(_1_itemsets)
        k = 2
        while True:
            k_itemsets = self.get_k_item_set(_1_itemsets, k)
            if len(k_itemsets) == 0:
                break
            self.frequent_itemsets.append(k_itemsets)
            _1_itemsets = k_itemsets
            self.update_dataset_by_deleting_infrequent_items(k_itemsets)
            k += 1
        return self.frequent_itemsets

    # update the dataset by deleting the infrequent items
    def update_dataset_by_deleting_infrequent_items(self, itemsets):
        new_dataset = []
        for itemset in itemsets:
            for transaction in self.dataset:
                if set(itemset).issubset(set(transaction)):
                    new_dataset.append(transaction)
                    break
        self.dataset = new_dataset
        print(len(self.dataset))
        return self.dataset
    
# test
apriori = Apriori(dataset, s)
frequent_item = apriori.get_1_item_set()
# print('frequent_item:', frequent_item)
# print('len(frequent_item):', len(frequent_item))
print('len(apriori.dataset):', len(apriori.dataset))
apriori.update_dataset(frequent_item)
print('len(apriori.dataset):', len(apriori.dataset))
# _2_itemsets = apriori.get_k_item_set(frequent_item, 2)
# print('len(_2_itemsets):', len(_2_itemsets))
fre_items = apriori.get_frequent_itemset()
print('len(fre_items):', len(fre_items))
print('fre_items:', fre_items)


len(apriori.dataset): 100000
len(apriori.dataset): 90993
2-itemset: (368, 368) support: 7828
2-itemset: (538, 538) support: 3982
2-itemset: (775, 775) support: 3771
2-itemset: (825, 825) support: 3085
2-itemset: (39, 39) support: 4258


In [ ]:
# get the association rules
def get_association_rules(frequent_itemsets):
    rules = []
    for itemset in frequent_itemsets:
        for item in itemset:
            for item_subset in list(itertools.combinations(itemset, len(itemset) - 1)):
                confidence = apriori.support(itemset) / apriori.support(item_subset)
                if confidence > 0.5:
                    rules.append((item_subset, item, confidence))
    return rules